# DEA Waterbodies all time statistics

In [1]:
%matplotlib inline
%matplotlib widget
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl

import sys
import glob
import itertools
import xarray as xr
import numpy as np
import pandas as pd

In [2]:
# Read in the csv
AttrTable = pd.read_csv('PaperData/dea_waterbodies_seasonal_area_ws.csv')

In [3]:
#create rectangle borders for no-data times (SLC-off only)
missingLSstart = '2012SUMMER'
missingLSend = '2013AUTUMN'

In [4]:
# Convert the columns we can to floats (from strings)
for column in AttrTable:
    try:
        AttrTable[column] = AttrTable[column].dtype(float)
    except:
        continue

In [5]:
AttrTable

,UID,STATE,Shape_Leng,Shape_Area,1986WINTER,1986SPRING,1987SUMMER,1987AUTUMN,1987WINTER,1987SPRING,...,2018AUTUMN,2018WINTER,2018SPRING,2019SUMMER,2019AUTUMN,2019WINTER,2019SPRING,2020SUMMER,2020AUTUMN,2020WINTER
0,q9cusmx7n,WA,5.500000e+02,1.187500e+04,0.0,NaN,NaN,0.0,NaN,0.000000e+00,...,5.089286e+03,6.250000e+03,6.562500e+03,7.053571e+03,7.083333e+03,7125.000000,5416.666667,5.491071e+03,6.875000e+03,10312.500000
1,q9cuu1dbv,WA,1.600000e+03,3.500000e+04,0.0,NaN,NaN,NaN,NaN,0.000000e+00,...,1.987500e+04,1.687500e+04,2.906250e+04,2.056250e+04,2.156250e+04,20520.833333,24375.000000,2.110577e+04,2.281250e+04,22500.000000
2,q9cuuc4dp,WA,4.000000e+02,5.000000e+03,0.0,NaN,NaN,0.0,NaN,0.000000e+00,...,2.291667e+03,1.696429e+03,1.625000e+03,1.964286e+03,2.500000e+03,2000.000000,1875.000000,1.979167e+03,2.000000e+03,1000.000000
3,q9cuv3u3x,WA,3.500000e+02,6.250000e+03,3750.0,NaN,NaN,NaN,NaN,5.000000e+03,...,3.750000e+03,2.291667e+03,3.541667e+03,4.187500e+03,3.750000e+03,2000.000000,3906.250000,3.076923e+03,3.125000e+03,3000.000000
4,q9cuv6j1r,WA,3.000000e+02,5.000000e+03,3125.0,NaN,NaN,NaN,NaN,4.375000e+03,...,1.875000e+03,1.770833e+03,7.812500e+02,1.015625e+03,1.125000e+03,2083.333333,1250.000000,9.375000e+02,1.750000e+03,1979.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295901,rnj90n5ef,QLD,1.050000e+03,5.062500e+04,NaN,NaN,NaN,NaN,NaN,9.375000e+03,...,4.187500e+04,2.218750e+04,2.500000e+03,NaN,3.250000e+04,38750.000000,28125.000000,2.687500e+04,2.062500e+04,NaN
295902,r42f3t8r0,SA,1.938015e+06,1.612169e+09,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.685000e+07,1.027083e+05,7.701000e+07,NaN,NaN,NaN,NaN,NaN
295903,qu0tbwjqr,WA,8.514678e+04,5.073126e+07,NaN,NaN,NaN,NaN,NaN,1.818750e+05,...,2.492917e+06,NaN,3.593750e+04,5.106875e+06,4.348958e+06,NaN,273625.000000,9.189375e+06,1.367042e+07,NaN
295904,r49b5nmwq,SA,9.934631e+04,3.878000e+07,NaN,NaN,NaN,NaN,NaN,5.768750e+05,...,NaN,1.860938e+07,2.110417e+05,4.387500e+04,6.005208e+06,NaN,5875.000000,1.184375e+06,0.000000e+00,NaN


## All Australia analysis

In [6]:
# Find the total number of waterbodies
NumberofWB = len(AttrTable)
print(NumberofWB)

# Find the total area of waterbodies
AreaofWB = AttrTable['Shape_Area'].sum()
print(AreaofWB)

295906
87548718812.30687


### Create an empty dataframe to put the summary statistics into

In [7]:
AttrTableStats = pd.DataFrame(columns=AttrTable.columns)

### Now calculate the number and area of wet/dry/unknown waterbody pixels

In [8]:
# Create a new row with just a count of waterbodies with some water in them
# we need to do this in a loop because of the string columns
for column in AttrTable:
    try:
        AttrTableStats.loc['TotalWetCount',column] = (AttrTable[column] > 0).sum()
    except:
        continue

# Create a new row called 'Total' that has the sums of each column
AttrTableStats.loc['TotalWetArea',:] = AttrTable.sum(axis=0)

# Create a new row with all the NaNs summed up
AttrTableStats.loc['TotalNaNCount',:] = AttrTable.isna().sum()

# Calculate the area of NaNs by multiplying the nan/not-nan boolean by the area of the waterbody
for column in AttrTable:
    try:
        AttrTableStats.loc['TotalNaNArea',column] = (AttrTable[column].isna() * AttrTable['Shape_Area']).sum()
    except TypeError:
        continue

# Create a new row with the remainder - aka dry
for column in AttrTable:
    try:
        AttrTableStats.loc['TotalDryArea',column] = AreaofWB - (AttrTableStats.loc['TotalWetArea'][column] + AttrTableStats.loc['TotalNaNArea'][column])
    except TypeError:
        continue

# Create a new row with the remainder as a total - aka dry
for column in AttrTable:
    try:
        AttrTableStats.loc['TotalDryCount',column] = NumberofWB - (AttrTableStats.loc['TotalWetCount'][column] + AttrTableStats.loc['TotalNaNCount'][column])
    except TypeError:
        continue

In [9]:
AttrTableStats

,UID,STATE,Shape_Leng,Shape_Area,1986WINTER,1986SPRING,1987SUMMER,1987AUTUMN,1987WINTER,1987SPRING,...,2018AUTUMN,2018WINTER,2018SPRING,2019SUMMER,2019AUTUMN,2019WINTER,2019SPRING,2020SUMMER,2020AUTUMN,2020WINTER
TotalWetCount,NaN,NaN,295906,295906,65604,5445,0,52981,51625,127630,...,210268,193785,179072,170210,199213,186304,159318,174456,223804,176024
TotalWetArea,q9cusmx7nq9cuu1dbvq9cuuc4dpq9cuv3u3xq9cuv6j1rq...,NaN,6.64782e+08,8.75487e+10,7.33521e+09,1.61153e+09,0,3.48095e+09,4.22842e+09,1.11469e+10,...,1.49238e+10,1.40228e+10,1.22592e+10,1.03431e+10,1.21154e+10,1.26873e+10,9.21084e+09,1.153e+10,1.70591e+10,1.14925e+10
TotalNaNCount,0,703,0,0,156411,280073,295906,149789,175724,9783,...,3262,5595,1062,4763,3499,6145,747,2736,2905,14363
TotalNaNArea,0,4.43894e+07,0,0,6.05009e+10,8.04157e+10,8.75487e+10,6.01655e+10,7.00617e+10,2.19771e+10,...,1.1301e+10,2.27875e+10,5.29117e+09,1.71508e+09,7.01703e+09,2.3219e+10,3.63842e+09,2.95595e+09,8.19617e+09,3.30288e+10
TotalDryArea,NaN,NaN,8.68839e+10,0,1.97126e+10,5.52149e+09,0,2.39023e+10,1.32586e+10,5.44248e+10,...,6.13239e+10,5.07385e+10,6.99984e+10,7.54905e+10,6.84163e+10,5.16424e+10,7.46995e+10,7.30627e+10,6.22934e+10,4.30274e+10
TotalDryCount,NaN,NaN,0,0,73891,10388,0,93136,68557,158493,...,82376,96526,115772,120933,93194,103457,135841,118714,69197,105519


### Create a stacked bar chart showing the relative area of water in each waterbody

In [10]:
DataToPlot = AttrTableStats.loc[['TotalWetArea', 'TotalDryArea', 'TotalNaNArea']]
DataToPlot.drop(['UID', 'STATE', 'Shape_Leng', 'Shape_Area'], axis = 1, inplace=True)
DataToPlot.loc['TotalNaNArea'] = DataToPlot.loc['TotalNaNArea'] / 1000000
DataToPlot.loc['TotalWetArea'] = DataToPlot.loc['TotalWetArea'] / 1000000
DataToPlot.loc['TotalDryArea'] = DataToPlot.loc['TotalDryArea'] / 1000000

DataMax = DataToPlot['1986WINTER'].sum()

In [11]:
DataToPlot

,1986WINTER,1986SPRING,1987SUMMER,1987AUTUMN,1987WINTER,1987SPRING,1988SUMMER,1988AUTUMN,1988WINTER,1988SPRING,...,2018AUTUMN,2018WINTER,2018SPRING,2019SUMMER,2019AUTUMN,2019WINTER,2019SPRING,2020SUMMER,2020AUTUMN,2020WINTER
TotalWetArea,7335.21,1611.53,0,3480.95,4228.42,11146.9,10934.7,15107.5,21862.4,12997.7,...,14923.8,14022.8,12259.2,10343.1,12115.4,12687.3,9210.84,11530,17059.1,11492.5
TotalDryArea,19712.6,5521.49,0,23902.3,13258.6,54424.8,64063.9,53693.8,49108.5,58709.5,...,61323.9,50738.5,69998.4,75490.5,68416.3,51642.4,74699.5,73062.7,62293.4,43027.4
TotalNaNArea,60500.9,80415.7,87548.7,60165.5,70061.7,21977.1,12550.2,18747.4,16577.8,15841.6,...,11301,22787.5,5291.17,1715.08,7017.03,23219,3638.42,2955.95,8196.17,33028.8


In [12]:
DataToPlot.columns[99]

'2011AUTUMN'

In [13]:
ax = DataToPlot.T.plot.bar(stacked=True, figsize=([15,5]), width = 1, color=['lightskyblue', 'sienna', 'lightgrey'], alpha=0.9);
ax.set_xticks(np.arange(0,len(ax.get_xticklabels()), 4));
ax.set_xticklabels([t[:4] for t in DataToPlot.columns[::4]], rotation=45);
plt.ylim([0, DataMax])
ax.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
plt.xlabel('Year', fontsize=14);
plt.ylabel('Total area (km$^2$)', fontsize = 14);
plt.legend(reversed(plt.legend().legendHandles), reversed(['Total wet area', 'Total dry area', 'Total unobserved area']))
plt.title('Seasonal average wet/dry/unobserved area totals for all Australian waterbodies', fontsize=16);

#set up rectangle
gap = DataToPlot.columns.get_loc(missingLSend) - DataToPlot.columns.get_loc(missingLSstart)
slc_rectangle = mpl.patches.Rectangle((DataToPlot.columns.get_loc(missingLSstart) - 0.5,0), gap, DataMax, alpha = 0.5, facecolor='white',
             edgecolor='white', hatch="////", linewidth=2)
ax.add_patch(slc_rectangle)

plt.savefig('AllAustraliaSeasonalObsAreaAllTime.jpg', bbox_inches= 'tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Now plot by count of waterbodies with some water in them (> 0)

In [12]:
DataToPlot = AttrTableStats.loc[['TotalWetCount', 'TotalDryCount', 'TotalNaNCount']]
DataToPlot.drop(['UID', 'STATE', 'Shape_Leng', 'Shape_Area'], axis = 1, inplace=True)

DataMax = DataToPlot['1986WINTER'].sum()

In [13]:
ax = DataToPlot.T.plot.bar(stacked=True, figsize=([15,5]), width = 1, color=['lightskyblue', 'sienna', 'lightgrey'], alpha=0.9);
ax.set_xticks(np.arange(0,len(ax.get_xticklabels()), 4));
ax.set_xticklabels([t[:4] for t in DataToPlot.columns[::4]], rotation=45);
plt.ylim([0, DataMax])
plt.xlabel('Year', fontsize=14);
plt.ylabel('Count', fontsize = 14);
plt.legend(reversed(plt.legend().legendHandles), reversed(['Number of wet waterbodies', 
                                                           'Number of dry waterbodies', 
                                                           'Number of unobserved waterbodies']))
plt.title('Total number of Australian waterbodies with at least some water observed in them', fontsize=16);

#set up rectangle
gap = DataToPlot.columns.get_loc(missingLSend) - DataToPlot.columns.get_loc(missingLSstart)
slc_rectangle = mpl.patches.Rectangle((DataToPlot.columns.get_loc(missingLSstart) - 0.5,0), gap, DataMax, alpha = 0.5, facecolor='white',
             edgecolor='white', hatch="////", linewidth=2)
ax.add_patch(slc_rectangle)

plt.savefig('AllAustraliaSeasonalObsCountAllTime.jpg', bbox_inches= 'tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## State by state analysis

In [ ]:
# Group the results by state
StateGroups = AttrTable.groupby('STATE').sum()
# Get rid of the lengh and area columns
StateGroups.drop(['Shape_Leng', 'Shape_Area'], axis = 1, inplace=True)
# Convert from m2 to km2
StateGroups = StateGroups / 1000000

In [ ]:
StateGroups

#### Create an empty dataframe for our percent statistics

In [ ]:
StateGroupsPct = pd.DataFrame(index=StateGroups.index, columns=StateGroups.columns)

#### Sum the wet area for all states for each quarter

In [ ]:
StateGroups.loc['TotalWetArea',:] = StateGroups.sum(axis=0)

#### Calculate the percentage of the total wet area observed in each state

In [ ]:
for row in StateGroupsPct.iterrows():
    StateGroupsPct.loc[row[0], :] = (StateGroups.loc[row[0]] / StateGroups.loc['TotalWetArea']) * 100

#### Sort the states by their average percentage of the total

In [ ]:
Order = StateGroupsPct.mean(axis=1).sort_values(ascending=False)

In [ ]:
StateGroupsPctSorted = StateGroupsPct.loc[Order.index]

#### Now plot!

In [ ]:
ax = StateGroupsPctSorted.T.plot.bar(stacked=True, figsize=([15,5]), width = 1, alpha=0.9);
ax.set_xticks(np.arange(0,len(ax.get_xticklabels()), 4));
ax.set_xticklabels([t[:4] for t in DataToPlot.columns[::4]], rotation=45);
plt.xlabel('Year', fontsize=14);
plt.ylabel('% of total observed wet area', fontsize = 14);
plt.legend(reversed(plt.legend().legendHandles), reversed(Order.index), loc='lower right', bbox_to_anchor=(1.1, 0.2));
plt.title('Percentage of total observed wet area by Australian state', fontsize=16);
plt.savefig('StatesSeasonalObsCountAllTime.jpg', bbox_inches= 'tight')

In [ ]:
StateGroupsSorted = StateGroups.loc[Order.index]

ax = StateGroupsSorted.T.plot.line(figsize=([15,5]));
ax.set_xticks(np.arange(0,len(ax.get_xticklabels()), 4));
ax.set_xticklabels([t[:4] for t in DataToPlot.columns[::4]], rotation=45);
plt.xlabel('Year', fontsize=14);
plt.ylabel('Wet area (km$^2$)', fontsize = 14);
plt.legend(reversed(plt.legend().legendHandles), reversed(Order.index), loc='lower right', bbox_to_anchor=(1.1, 0.2));
plt.title('Total observed wet area by Australian state', fontsize=16);
# plt.savefig('AllAustraliaSeasonalObsCountAllTime.jpg')

In [ ]:
StateGroupsDrop = StateGroups.iloc[:, 5:]

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=([18,6]), sharex=True);
# Western Australia
DataMax = (StateGroupsDrop.loc['WA'].max() / 100) * 108
axs[0, 0].plot(StateGroupsDrop.loc['WA'], 'k');
axs[0, 0].set_title('Western Australia');
axs[0, 0].set_ylabel('Wet area (km$^2$)')
axs[0, 0].set_ylim([0, DataMax])
#set up rectangle
gap = StateGroupsDrop.columns.get_loc(missingLSend) - StateGroupsDrop.columns.get_loc(missingLSstart)
slc_rectangle = mpl.patches.Rectangle((StateGroupsDrop.columns.get_loc(missingLSstart) - 0.5, axs[0, 0].get_ylim()[0] + 1), 
                                      gap, DataMax, alpha = 0.5, facecolor='white',
                                      edgecolor='grey', hatch="////", linewidth=1)
axs[0, 0].add_patch(slc_rectangle)

# Northern Territory
DataMax = (StateGroupsDrop.loc['NT'].max() / 100) * 108
axs[0, 1].plot(StateGroupsDrop.loc['NT'], 'k');
axs[0, 1].set_title('Northern Territory');
axs[0, 1].set_ylim([0, DataMax])
#set up rectangle
gap = StateGroupsDrop.columns.get_loc(missingLSend) - StateGroupsDrop.columns.get_loc(missingLSstart)
slc_rectangle = mpl.patches.Rectangle((StateGroupsDrop.columns.get_loc(missingLSstart) - 0.5, axs[0, 1].get_ylim()[0]), 
                                      gap, DataMax, alpha = 0.5, facecolor='white',
                                      edgecolor='grey', hatch="////", linewidth=1)
axs[0, 1].add_patch(slc_rectangle)

# Queensland
DataMax = (StateGroupsDrop.loc['QLD'].max() / 100) * 108
axs[0, 2].plot(StateGroupsDrop.loc['QLD'], 'k');
axs[0, 2].set_title('Queensland');
axs[0, 2].set_ylim([0, DataMax])
#set up rectangle
gap = StateGroupsDrop.columns.get_loc(missingLSend) - StateGroupsDrop.columns.get_loc(missingLSstart)
slc_rectangle = mpl.patches.Rectangle((StateGroupsDrop.columns.get_loc(missingLSstart) - 0.5, axs[0, 2].get_ylim()[0]), 
                                      gap, DataMax, alpha = 0.5, facecolor='white',
                                      edgecolor='grey', hatch="////", linewidth=1)
axs[0, 2].add_patch(slc_rectangle)

# New South Wales
DataMax = (StateGroupsDrop.loc['NSW'].max() / 100) * 108
axs[1, 0].plot(StateGroupsDrop.loc['NSW'], 'k');
axs[1, 0].set_title('New South Wales');
axs[1, 0].set_ylabel('Wet area (km$^2$)')
axs[1, 0].set_xlabel('Year');
#set up rectangle
gap = StateGroupsDrop.columns.get_loc(missingLSend) - StateGroupsDrop.columns.get_loc(missingLSstart)
slc_rectangle = mpl.patches.Rectangle((StateGroupsDrop.columns.get_loc(missingLSstart) - 0.5, axs[1, 0].get_ylim()[0]), 
                                      gap, DataMax, alpha = 0.5, facecolor='white',
                                      edgecolor='grey', hatch="////", linewidth=1)
axs[1, 0].add_patch(slc_rectangle)

# Victoria
DataMax = (StateGroupsDrop.loc['VIC'].max() / 100) * 108
axs[1, 1].plot(StateGroupsDrop.loc['VIC'], 'k');
axs[1, 1].set_title('Victoria');
axs[1, 1].set_xlabel('Year');
#set up rectangle
gap = StateGroupsDrop.columns.get_loc(missingLSend) - StateGroupsDrop.columns.get_loc(missingLSstart)
slc_rectangle = mpl.patches.Rectangle((StateGroupsDrop.columns.get_loc(missingLSstart) - 0.5, axs[1, 1].get_ylim()[0]), 
                                      gap, DataMax, alpha = 0.5, facecolor='white',
                                      edgecolor='grey', hatch="////", linewidth=1)
axs[1, 1].add_patch(slc_rectangle)

# Tasmania
DataMax = (StateGroupsDrop.loc['TAS'].max() / 100) * 108
axs[1, 2].plot(StateGroupsDrop.loc['TAS'], 'k');
axs[1, 2].set_title('Tasmania');
axs[1, 2].set_xlabel('Year');
#set up rectangle
gap = StateGroupsDrop.columns.get_loc(missingLSend) - StateGroupsDrop.columns.get_loc(missingLSstart)
slc_rectangle = mpl.patches.Rectangle((StateGroupsDrop.columns.get_loc(missingLSstart) - 0.5, axs[1, 2].get_ylim()[0]), 
                                      gap, DataMax, alpha = 0.5, facecolor='white',
                                      edgecolor='grey', hatch="////", linewidth=1)
axs[1, 2].add_patch(slc_rectangle)

plt.setp(axs, xticks=np.arange(0,len(axs[1,0].get_xticklabels()), 11), xticklabels=[t[:4] for t in StateGroupsDrop.columns[::11]]);
axs[1, 0].set_xticklabels([t[:4] for t in StateGroupsDrop.columns[::11]], rotation=45);
axs[1, 1].set_xticklabels([t[:4] for t in StateGroupsDrop.columns[::11]], rotation=45);
axs[1, 2].set_xticklabels([t[:4] for t in StateGroupsDrop.columns[::11]], rotation=45);
fig.suptitle('Total observed wet area by Australian state', fontsize=16);
plt.savefig('StatesSeasonalObsCountAllTimeLine.jpg', bbox_inches= 'tight')